# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [23]:
%pylab inline
import json
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


ROBOT_POSE_DATA_DIR = 'robot_pose_data_json'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\thekl\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = json.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [5]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(0, len(classes)):
    data, target = load_pose_data(i)
    all_data = all_data +  data
    all_target = all_target + target

print('total number of data', len(all_data))

total number of data 222


In [12]:
# shuffle data
permutation = np.random.permutation(len(all_data))
shuffle_all_data = []
shuffle_all_target = []

#for i in range(0, len(permutation)):
#    shuffle_all_data[i] = all_data[permuation[i]]
#    shuffle_all_target[i] = all_target[permuation[i]]

#all_data = [all_data[i] for i in permutation]
#all_target = [all_target[i] for i in permutation]

    
for p in permutation:
    shuffle_all_data.append(all_data[p])
    shuffle_all_target.append(all_target[p])

n = int(len(shuffle_all_data)*0.5)

training_data = shuffle_all_data[0:n]
learn_data = shuffle_all_data[n:len(shuffle_all_data)]

training_target = shuffle_all_target[0:n]
learn_target = shuffle_all_target[n:len(shuffle_all_target)]

[[4.1961669921875e-05,
  -0.007627964019775391,
  -0.2530679702758789,
  0.9004161357879639,
  4.1961669921875e-05,
  -0.007627964019775391,
  -0.5397986173629761,
  0.9572582244873047,
  0.00575273809954524,
  -0.0164906345307827],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.5707963267948966],
 [0.14730596542358398,
  0.6796040534973145,
  -1.4189081192016602,
  -0.11202406883239746,
  0.14730596542358398,
  0.3697359561920166,
  -0.7332940101623535,
  0.052197933197021484,
  -1.3997540473937988,
  -0.06981325149536133],
 [0.3037738800048828,
  0.015382050536572933,
  0.42035794258117676,
  1.9818861484527588,
  0.3037738800048828,
  -0.01683211326599121,
  0.5199840068817139,
  1.7119860649108887,
  0.0,
  -0.08726692199707031],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.5707963267948966],
 [0.004643917083740234,
  -0.033705949783325195,
  -0.9970581531524658,
  2.1414220333099365,
  0.004643917083740234,
  0.029187917709350586,
  -0.995607852935791,
  2.1415061950683594

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

### learning

In [16]:
X = learn_data
y = learn_target
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

### predicting

In [18]:
clf.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)

In [19]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [ ]:
expected = []
predicted = []

expected = training_target
predicted = clf.predict(training_data)

evaluate(expected, predicted)

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [21]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [26]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)